Machine Learning

In [1]:
import requests
import pickle
import pandas as pd
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

In [2]:
import pickle

with open('match_df', 'rb') as f:
    match_df = pickle.load(f)

In [3]:
#matches_df = pd.read_csv("match.csv", index_col=0)

In [4]:
matches = match_df.reset_index()
del matches['index']
matches.index = matches.index + 1


In [5]:
matches["verein"].value_counts()

Bayer Leverkusen       63
Bayern Munich          63
Koln                   63
Mainz 05               63
Bochum                 63
Wolfsburg              63
Union Berlin           63
Werder Bremen          63
Monchengladbach        63
Hoffenheim             63
Freiburg               63
Augsburg               63
Eintracht Frankfurt    63
Dortmund               63
RB Leipzig             63
Stuttgart              63
Schalke 04             34
Hertha BSC             34
Heidenheim             29
Darmstadt 98           29
Name: verein, dtype: int64

In [6]:
matches.dtypes

datum              object
uhrzeit            object
wett               object
runde              object
tag                object
spielort           object
ergebnis           object
tf                 object
tk                 object
gegner             object
xg                float64
xga               float64
besitz            float64
zuschauerzahl     float64
kapitän            object
aufstellung        object
schiedsrichter     object
spielbericht       object
hinweise           object
sc                float64
sat               float64
entf.             float64
fs                float64
elf                 int64
veelf               int64
saison              int64
verein             object
dtype: object

In [7]:
matches["datum"] = pd.to_datetime(matches["datum"],dayfirst=True)


In [8]:
matches["venue_code"] = matches["spielort"].astype("category").cat.codes

In [9]:
matches["opp_code"] = matches["gegner"].astype("category").cat.codes

In [10]:
#matches['datum'] = pd.to_datetime(matches['datum'])


In [11]:
matches2 = matches[(matches['datum'] > '2024-04-10') & (matches['datum'] < '2024-04-25')]

In [12]:
matches = matches[~(matches['datum'] > '2024-02-27')]


In [13]:
matches["time"] = matches["uhrzeit"].str.replace(":.+", "", regex=True).astype("int")

In [14]:
matches["day_code"] = matches["datum"].dt.dayofweek

In [15]:
def transform(outcome):
    if outcome == "N":
        return 0
    elif outcome == "U":
        return 1
    elif outcome == "S":
        return 2

    

In [16]:
matches["target"] = matches["ergebnis"].apply(transform)

In [17]:
matches["target"] = matches["target"].astype(int)

Random Forest

In [18]:
rf = RandomForestClassifier

In [19]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [20]:
train = matches[matches["datum"]< "2024-01-01"]

In [21]:
test = matches[matches["datum"]> "2024-01-01"]

In [22]:
predictors = ["venue_code", "opp_code","time"]

In [23]:
rf.fit(train[predictors],train["target"])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [24]:
preds = rf.predict(test[predictors])

In [25]:
acc = accuracy_score(test["target"], preds)

In [26]:
acc

0.4140625

In [27]:
combined = pd.DataFrame(dict(actual=test["target"], prediction = preds))

In [28]:
pd.crosstab(index=combined["actual"], columns=combined["prediction"])

prediction,0,1,2
actual,,,
0,26,2,13
1,18,5,23
2,12,7,22


In [29]:
precision_score(test["target"],preds,average='micro')

0.4140625

Next Part

In [30]:
grouped_matches = matches.groupby("verein")

In [31]:
group = grouped_matches.get_group("Bayer Leverkusen")

In [32]:
group.head(5)

,datum,uhrzeit,wett,runde,tag,spielort,ergebnis,tf,tk,gegner,...,fs,elf,veelf,saison,verein,venue_code,opp_code,time,day_code,target
1,2023-08-19,15:30,Bundesliga,Spielwoche 1,Sa.,Heim,S,3.0,2.0,RB Leipzig,...,0.0,0,0,2024,Bayer Leverkusen,1,14,15,5,2
2,2023-08-26,18:30,Bundesliga,Spielwoche 2,Sa.,Auswärts,S,3.0,0.0,M'Gladbach,...,0.0,0,0,2024,Bayer Leverkusen,0,12,18,5,2
3,2023-09-02,15:30,Bundesliga,Spielwoche 3,Sa.,Heim,S,5.0,1.0,Darmstadt 98,...,1.0,0,0,2024,Bayer Leverkusen,1,3,15,5,2
4,2023-09-15,20:30,Bundesliga,Spielwoche 4,Fr.,Auswärts,U,2.0,2.0,Bayern München,...,1.0,1,1,2024,Bayer Leverkusen,0,1,20,4,1
5,2023-09-24,15:30,Bundesliga,Spielwoche 5,So.,Heim,S,4.0,1.0,Heidenheim,...,0.0,1,1,2024,Bayer Leverkusen,1,7,15,6,2


In [33]:
def rolling_averages(group,cols, new_cols):
    group = group.sort_values("datum")
    rolling_stats = group[cols].rolling(3, closed="left").mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [34]:
cols = ["tf", "tk", "sc", "sat", "entf.", "fs", "elf", "veelf"]
new_cols = [f"{c}_rolling" for c in cols]

In [35]:
new_cols

['tf_rolling',
 'tk_rolling',
 'sc_rolling',
 'sat_rolling',
 'entf._rolling',
 'fs_rolling',
 'elf_rolling',
 'veelf_rolling']

In [36]:
rolling_averages(group, cols, new_cols)

,datum,uhrzeit,wett,runde,tag,spielort,ergebnis,tf,tk,gegner,...,day_code,target,tf_rolling,tk_rolling,sc_rolling,sat_rolling,entf._rolling,fs_rolling,elf_rolling,veelf_rolling
696,2022-08-27,15:30,Bundesliga,Spielwoche 4,Sa.,Auswärts,S,3,0,Mainz 05,...,5,2,0.333333,2.000000,15.333333,5.333333,189.000000,1.333333,0.000000,0.000000
697,2022-09-03,15:30,Bundesliga,Spielwoche 5,Sa.,Heim,N,2,3,Freiburg,...,5,0,1.333333,1.666667,15.333333,6.000000,185.000000,1.666667,0.000000,0.000000
698,2022-09-10,15:30,Bundesliga,Spielwoche 6,Sa.,Auswärts,U,2,2,Hertha BSC,...,5,1,1.666667,2.000000,12.333333,4.666667,180.333333,1.333333,0.000000,0.000000
699,2022-09-17,15:30,Bundesliga,Spielwoche 7,Sa.,Heim,U,1,1,Werder Bremen,...,5,1,2.333333,1.666667,11.333333,5.333333,171.000000,0.666667,0.000000,0.000000
700,2022-09-30,20:30,Bundesliga,Spielwoche 8,Fr.,Auswärts,N,0,4,Bayern München,...,4,0,1.666667,2.000000,15.000000,6.000000,158.000000,0.333333,0.000000,0.000000
701,2022-10-08,15:30,Bundesliga,Spielwoche 9,Sa.,Heim,S,4,0,Schalke 04,...,5,2,1.000000,2.333333,13.333333,5.666667,164.666667,0.333333,0.000000,0.000000
702,2022-10-15,15:30,Bundesliga,Spielwoche 10,Sa.,Auswärts,N,1,5,Eint Frankfurt,...,5,0,1.666667,1.666667,14.000000,6.333333,158.333333,0.000000,0.000000,0.000000
703,2022-10-22,15:30,Bundesliga,Spielwoche 11,Sa.,Heim,U,2,2,Wolfsburg,...,5,1,1.666667,3.000000,9.333333,4.666667,159.333333,0.000000,0.000000,0.000000
704,2022-10-29,15:30,Bundesliga,Spielwoche 12,Sa.,Auswärts,N,0,2,RB Leipzig,...,5,0,2.333333,2.333333,10.000000,4.666667,172.666667,0.000000,0.000000,0.333333
705,2022-11-06,15:30,Bundesliga,Spielwoche 13,So.,Heim,S,5,0,Union Berlin,...,6,2,1.000000,3.000000,7.333333,2.666667,185.333333,0.000000,0.000000,0.333333


In [37]:
matches_rolling = matches.groupby("verein").apply(lambda x: rolling_averages(x,cols,new_cols))

In [38]:
matches_rolling = matches_rolling.droplevel("verein")

In [39]:
matches_rolling.index = range(matches_rolling.shape[0])

In [40]:
def make_predictions(data, predictors):
    train = data[data["datum"]< "2024-01-01"]
    test = data[data["datum"]> "2024-01-01"]
    rf.fit(train[predictors],train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted = preds), index = test.index)
    precision = precision_score(test["target"],preds,average='micro')
    return combined, precision

In [41]:
combined, precision = make_predictions(matches_rolling, predictors + new_cols)

In [42]:
precision

0.375

In [43]:
combined

,actual,predicted
47,0,2
48,2,0
49,0,2
50,1,0
51,1,2
...,...,...
961,1,2
962,1,2
963,0,0
964,1,2


In [44]:
pd.set_option('display.max_columns', 40)


In [45]:
matches_rolling

,datum,uhrzeit,wett,runde,tag,spielort,ergebnis,tf,tk,gegner,xg,xga,besitz,zuschauerzahl,kapitän,aufstellung,schiedsrichter,spielbericht,hinweise,sc,sat,entf.,fs,elf,veelf,saison,verein,venue_code,opp_code,time,day_code,target,tf_rolling,tk_rolling,sc_rolling,sat_rolling,entf._rolling,fs_rolling,elf_rolling,veelf_rolling
0,2022-08-27,15:30,Bundesliga,Spielwoche 4,Sa.,Auswärts,N,0,1,Hoffenheim,0.6,1.9,50.0,17.206,Jeffrey Gouweleeuw,3-4-1-2,Tobias Welz,Spielbericht,NaN,11.0,1.0,171.0,0.0,0,0,2023,Augsburg,0,9,15,5,0,1.000000,2.333333,8.000000,2.666667,194.333333,0.666667,0.000000,0.000000
1,2022-09-04,15:30,Bundesliga,Spielwoche 5,So.,Heim,N,0,2,Hertha BSC,0.4,2.1,48.0,25.789,Jeffrey Gouweleeuw,3-5-2,Harm Osmers,Spielbericht,NaN,7.0,1.0,216.0,1.0,0,0,2023,Augsburg,1,8,15,6,0,1.000000,1.333333,8.000000,2.333333,193.333333,0.333333,0.000000,0.000000
2,2022-09-09,20:30,Bundesliga,Spielwoche 6,Fr.,Auswärts,S,1,0,Werder Bremen,1.0,1.3,32.0,41.000,Jeffrey Gouweleeuw,3-4-3,Martin Petersen,Spielbericht,NaN,15.0,6.0,225.0,0.0,0,0,2023,Augsburg,0,18,20,4,2,0.333333,1.666667,8.333333,1.333333,202.000000,0.666667,0.000000,0.000000
3,2022-09-17,15:30,Bundesliga,Spielwoche 7,Sa.,Heim,S,1,0,Bayern München,1.7,1.6,24.0,30.660,Jeffrey Gouweleeuw,4-2-3-1,Marco Fritz,Spielbericht,NaN,10.0,4.0,129.0,0.0,0,0,2023,Augsburg,1,1,15,5,2,0.333333,1.000000,11.000000,2.666667,204.000000,0.333333,0.000000,0.000000
4,2022-10-02,17:30,Bundesliga,Spielwoche 8,So.,Auswärts,S,3,2,Schalke 04,1.4,1.8,44.0,60.328,Jeffrey Gouweleeuw,4-2-2-2,Daniel Schlager,Spielbericht,NaN,12.0,6.0,176.0,0.0,0,0,2023,Augsburg,0,15,17,6,2,0.666667,0.666667,10.666667,3.666667,190.000000,0.333333,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
961,2024-01-27,15:30,Bundesliga,Spielwoche 19,Sa.,Heim,U,1.0,1.0,Köln,1.5,0.6,52.0,24.525,Koen Casteels,4-2-3-1,Sören Storks,Spielbericht,NaN,14.0,4.0,111.0,1.0,0,0,2024,Wolfsburg,1,10,15,5,1,1.000000,1.333333,9.000000,3.666667,179.333333,0.000000,0.000000,0.000000
962,2024-02-04,15:30,Bundesliga,Spielwoche 20,So.,Heim,U,2.0,2.0,Hoffenheim,1.7,1.7,45.0,22.917,Maximilian Arnold,4-4-2,Florian Badstübner,Spielbericht,NaN,13.0,6.0,164.0,0.0,1,1,2024,Wolfsburg,1,9,15,6,1,1.000000,1.000000,12.000000,3.666667,146.333333,0.333333,0.000000,0.000000
963,2024-02-10,15:30,Bundesliga,Spielwoche 21,Sa.,Auswärts,N,0.0,1.0,Union Berlin,1.3,0.5,58.0,22.000,Maximilian Arnold,4-4-2,Matthias Jöllenbeck,Spielbericht,NaN,17.0,4.0,165.0,1.0,0,0,2024,Wolfsburg,0,17,15,5,0,1.333333,1.333333,13.333333,4.333333,147.000000,0.333333,0.333333,0.333333
964,2024-02-17,15:30,Bundesliga,Spielwoche 22,Sa.,Heim,U,1.0,1.0,Dortmund,1.8,1.4,37.0,28.917,Maximilian Arnold,4-2-3-1,Martin Petersen,Spielbericht,NaN,11.0,4.0,153.0,0.0,0,0,2024,Wolfsburg,1,4,15,5,1,1.000000,1.333333,14.666667,4.666667,146.666667,0.666667,0.333333,0.333333


In [46]:
combined = combined.merge(matches_rolling[["datum", "verein", "gegner", "ergebnis"]], left_index=True, right_index=True)

In [47]:
combined

,actual,predicted,datum,verein,gegner,ergebnis
47,0,2,2024-01-13,Augsburg,Leverkusen,N
48,2,0,2024-01-21,Augsburg,M'Gladbach,S
49,0,2,2024-01-27,Augsburg,Bayern München,N
50,1,0,2024-02-03,Augsburg,Bochum,U
51,1,2,2024-02-10,Augsburg,RB Leipzig,U
...,...,...,...,...,...,...
961,1,2,2024-01-27,Wolfsburg,Köln,U
962,1,2,2024-02-04,Wolfsburg,Hoffenheim,U
963,0,0,2024-02-10,Wolfsburg,Union Berlin,N
964,1,2,2024-02-17,Wolfsburg,Dortmund,U


In [48]:
combined["verein"].unique()

array(['Augsburg', 'Bayer Leverkusen', 'Bayern Munich', 'Bochum',
       'Darmstadt 98', 'Dortmund', 'Eintracht Frankfurt', 'Freiburg',
       'Heidenheim', 'Hoffenheim', 'Koln', 'Mainz 05', 'Monchengladbach',
       'RB Leipzig', 'Stuttgart', 'Union Berlin', 'Werder Bremen',
       'Wolfsburg'], dtype=object)

In [49]:
combined["gegner"].unique()

array(['Leverkusen', "M'Gladbach", 'Bayern München', 'Bochum',
       'RB Leipzig', 'Mainz 05', 'Freiburg', 'Augsburg', 'Darmstadt 98',
       'Heidenheim', 'Hoffenheim', 'Werder Bremen', 'Union Berlin',
       'Stuttgart', 'Dortmund', 'Eint Frankfurt', 'Köln', 'Wolfsburg'],
      dtype=object)

In [50]:
class MissingDict(dict):
    __missing__ = lambda self, key: key

map_values = {
    "Bayer Leverkusen":'Leverkusen',
    'Bayern Munich': 'Bayern München',
    'Eintracht Frankfurt': 'Eint Frankfurt',
    'Koln': 'Köln',
    'Monchengladbach': "M'Gladbach"
}
mapping = MissingDict(**map_values)


In [51]:
combined["new_team"] = combined["verein"].map(mapping)

In [52]:
merged = combined.merge(combined, left_on=["datum","new_team"],right_on=["datum","gegner"])

In [53]:
merged

,actual_x,predicted_x,datum,verein_x,gegner_x,ergebnis_x,new_team_x,actual_y,predicted_y,verein_y,gegner_y,ergebnis_y,new_team_y
0,0,2,2024-01-13,Augsburg,Leverkusen,N,Augsburg,2,0,Bayer Leverkusen,Augsburg,S,Leverkusen
1,2,0,2024-01-21,Augsburg,M'Gladbach,S,Augsburg,0,0,Monchengladbach,Augsburg,N,M'Gladbach
2,0,2,2024-01-27,Augsburg,Bayern München,N,Augsburg,2,0,Bayern Munich,Augsburg,S,Bayern München
3,1,0,2024-02-03,Augsburg,Bochum,U,Augsburg,1,0,Bochum,Augsburg,U,Bochum
4,1,2,2024-02-10,Augsburg,RB Leipzig,U,Augsburg,1,0,RB Leipzig,Augsburg,U,RB Leipzig
...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,1,2,2024-01-27,Wolfsburg,Köln,U,Wolfsburg,1,0,Koln,Wolfsburg,U,Köln
124,1,2,2024-02-04,Wolfsburg,Hoffenheim,U,Wolfsburg,1,0,Hoffenheim,Wolfsburg,U,Hoffenheim
125,0,0,2024-02-10,Wolfsburg,Union Berlin,N,Wolfsburg,2,2,Union Berlin,Wolfsburg,S,Union Berlin
126,1,2,2024-02-17,Wolfsburg,Dortmund,U,Wolfsburg,1,0,Dortmund,Wolfsburg,U,Dortmund


In [54]:
#merged[(merged["predicted_x"] == 2)& (merged["predicted_y"] == 0 )]["actual_x"].value_counts()

In [55]:
#28/49

In [56]:
merged.head(50)

,actual_x,predicted_x,datum,verein_x,gegner_x,ergebnis_x,new_team_x,actual_y,predicted_y,verein_y,gegner_y,ergebnis_y,new_team_y
0,0,2,2024-01-13,Augsburg,Leverkusen,N,Augsburg,2,0,Bayer Leverkusen,Augsburg,S,Leverkusen
1,2,0,2024-01-21,Augsburg,M'Gladbach,S,Augsburg,0,0,Monchengladbach,Augsburg,N,M'Gladbach
2,0,2,2024-01-27,Augsburg,Bayern München,N,Augsburg,2,0,Bayern Munich,Augsburg,S,Bayern München
3,1,0,2024-02-03,Augsburg,Bochum,U,Augsburg,1,0,Bochum,Augsburg,U,Bochum
4,1,2,2024-02-10,Augsburg,RB Leipzig,U,Augsburg,1,0,RB Leipzig,Augsburg,U,RB Leipzig
5,0,0,2024-02-17,Augsburg,Mainz 05,N,Augsburg,2,2,Mainz 05,Augsburg,S,Mainz 05
6,2,2,2024-02-25,Augsburg,Freiburg,S,Augsburg,0,0,Freiburg,Augsburg,N,Freiburg
7,2,0,2024-01-13,Bayer Leverkusen,Augsburg,S,Leverkusen,0,2,Augsburg,Leverkusen,N,Augsburg
8,2,2,2024-01-20,Bayer Leverkusen,RB Leipzig,S,Leverkusen,0,2,RB Leipzig,Leverkusen,N,RB Leipzig
9,1,2,2024-01-27,Bayer Leverkusen,M'Gladbach,U,Leverkusen,1,0,Monchengladbach,Leverkusen,U,M'Gladbach


Insert new Data


In [57]:
matches_predict = matches2

In [58]:
matches_predict["time"] = matches_predict["uhrzeit"].str.replace(":.+", "", regex=True).astype("int")

In [59]:
matches_predict["day_code"] = matches["datum"].dt.dayofweek

In [60]:
matches_predict["target"] = matches["ergebnis"].apply(transform)

In [61]:
matches_predict["target"] = matches["target"].astype(int)

In [62]:
matches_predict.head()

,datum,uhrzeit,wett,runde,tag,spielort,ergebnis,tf,tk,gegner,xg,xga,besitz,zuschauerzahl,kapitän,aufstellung,schiedsrichter,spielbericht,hinweise,sc,sat,entf.,fs,elf,veelf,saison,verein,venue_code,opp_code,time,day_code,target
29,2024-04-14,17:30,Bundesliga,Spielwoche 29,So.,Heim,S,5.0,0.0,Werder Bremen,3.2,0.7,56.0,30.210,Lukáš Hrádecký,3-4-3,Harm Osmers,Spielbericht,NaN,19.0,9.0,162.0,0.0,1,1,2024,Bayer Leverkusen,1,18,17,NaN,NaN
58,2024-04-13,15:30,Bundesliga,Spielwoche 29,Sa.,Heim,S,2.0,0.0,Köln,2.2,0.9,69.0,75.000,Thomas Müller,4-2-3-1,Frank Willenborg,Spielbericht,NaN,16.0,6.0,149.0,0.0,0,0,2024,Bayern Munich,1,10,15,NaN,NaN
87,2024-04-13,18:30,Bundesliga,Spielwoche 29,Sa.,Heim,S,3.0,0.0,Eint Frankfurt,1.2,0.6,54.0,60.000,Waldemar Anton,4-2-3-1,Deniz Aytekin,Spielbericht,NaN,9.0,6.0,163.0,0.0,0,0,2024,Stuttgart,1,5,18,NaN,NaN
116,2024-04-13,15:30,Bundesliga,Spielwoche 29,Sa.,Heim,S,3.0,0.0,Wolfsburg,1.3,0.4,50.0,45.858,Willi Orban,4-2-2-2,Christian Dingert,Spielbericht,NaN,16.0,6.0,150.0,0.0,0,0,2024,RB Leipzig,1,19,15,NaN,NaN
145,2024-04-13,15:30,Bundesliga,Spielwoche 29,Sa.,Auswärts,S,2.0,1.0,M'Gladbach,1.3,1.1,54.0,54.042,Gregor Kobel,4-2-3-1,Florian Badstübner,Spielbericht,NaN,11.0,3.0,219.0,0.0,1,1,2024,Dortmund,0,12,15,NaN,NaN


In [63]:
# Assuming 'rf' is your trained RandomForest model
if hasattr(rf, 'feature_names_in_'):
    print("Features used in training:")
    for feature in rf.feature_names_in_:
        print(feature)
else:
    print("The model does not have the 'feature_names_in_' attribute. Ensure it was trained with a pandas DataFrame.")

Features used in training:
venue_code
opp_code
time
tf_rolling
tk_rolling
sc_rolling
sat_rolling
entf._rolling
fs_rolling
elf_rolling
veelf_rolling


In [64]:
matches_predict2 = matches_predict.reindex(columns = matches_predict.columns.tolist() + ['tf_rolling',
 'tk_rolling',
 'sc_rolling',
 'sat_rolling',
 'entf._rolling',
 'fs_rolling',
 'elf_rolling',
 'veelf_rolling'])


In [65]:
columns_to_keep = ["venue_code", "opp_code","time", "tf_rolling","tk_rolling","sc_rolling","sat_rolling","entf._rolling","fs_rolling","elf_rolling","veelf_rolling"]


In [66]:
matches_predict4 = matches_predict2[columns_to_keep]

In [67]:
matches_predict4 = matches_predict4.fillna(0)

In [68]:
predictions = rf.predict(matches_predict4)

In [69]:
predictions

array([2, 2, 2, 2, 0, 1, 2, 1, 1, 1, 2, 1, 1, 0, 2, 2, 1, 2])

In [70]:
matches_predict2['predictions'] = predictions


In [71]:
df_reduziert = matches_predict2[["datum",	"uhrzeit", "gegner",  	"verein","predictions"]]


In [72]:
df_reduziert


,datum,uhrzeit,gegner,verein,predictions
29,2024-04-14,17:30,Werder Bremen,Bayer Leverkusen,2
58,2024-04-13,15:30,Köln,Bayern Munich,2
87,2024-04-13,18:30,Eint Frankfurt,Stuttgart,2
116,2024-04-13,15:30,Wolfsburg,RB Leipzig,2
145,2024-04-13,15:30,M'Gladbach,Dortmund,0
174,2024-04-13,18:30,Stuttgart,Eintracht Frankfurt,1
203,2024-04-12,20:30,Union Berlin,Augsburg,2
232,2024-04-14,15:30,Darmstadt 98,Freiburg,1
261,2024-04-13,15:30,Mainz 05,Hoffenheim,1
290,2024-04-13,15:30,Bochum,Heidenheim,1
